In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    BitsAndBytesConfig,
    TrainingArguments,
)

'''model_name = "NousResearch/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)'''

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-24 14:37:02,699] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


'model_name = "NousResearch/Llama-2-7b-chat-hf"\ncompute_dtype = getattr(torch, "float16")\n\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=compute_dtype\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_name,\n    quantization_config=bnb_config,\n    use_cache=False,\n    device_map="auto"\n)'

In [2]:
import torch
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoTokenizer

In [9]:
import deepspeed
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

base_dir = "llama-linear-layers"
checkpoint_number = "checkpoint-200"
checkpoint_path = "{}/{}".format(base_dir, checkpoint_number)

#peft_model = PeftModel.from_pretrained(model, checkpoint_path)

# load base LLM model and tokenizer
peft_model = AutoPeftModelForCausalLM.from_pretrained(
    checkpoint_path,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
#re_tokenizer = AutoTokenizer.from_pretrained(base_dir)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.84s/it]


In [10]:
config = {
    #"kernel_inject": True,
    #"tensor_parallel": {"tp_size": 4},
    "dtype": "fp16",
    #"enable_cuda_graph": False
}

ds_engine = deepspeed.init_inference(peft_model, config=config)
deepspeed_peft_model = ds_engine.module

[2023-11-24 14:42:45,691] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.12.3, git-hash=unknown, git-branch=unknown
[2023-11-24 14:42:45,694] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [11]:
deepspeed_peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(
                (lora_dropout): ModuleD

In [12]:
restored_tokenizer = AutoTokenizer.from_pretrained(checkpoint_path) #checkpoint_path #model_name

In [13]:
import pandas as pd
import time

eval_conv = pd.read_csv("../data/eval_dataset.csv", sep="\t")
eval_conv

,conversations,tokens
0,\n\n<s>[INST] I just set up a fresh galaxy ins...,202
1,\n\n<s>[INST] Dear everyone!\nI am currently a...,504
2,"\n\n<s>[INST] Really just a minor nuisance, bu...",153
3,\n\n<s>[INST] I would like someone to explain ...,104
4,"\n\n<s>[INST] Hi everyone,\nI am running a UMI...",462
5,\n\n<s>[INST] In my galaxy.yml I have:\nuser_a...,113
6,"\n\n<s>[INST] Hi,\nI’d like to remove all line...",134
7,\n\n<s>[INST] The history window has frequent ...,186
8,\n\n<s>[INST] I want to upload the fastQ data ...,409
9,"\n\n<s>[INST] Hello, I’m trying to run stringt...",732


In [14]:
predictions = []
original_instructions = []
ground_truth_answer = []
extracted_answers = []

start_marker = '<s>[INST]'
end_marker = '[/INST]'
end_tag = "</s>"

s_time = time.time()

for ri, row in eval_conv.iterrows():
    entire_conv = row["conversations"]
    start_index = entire_conv.find(start_marker)
    end_index = entire_conv.find(end_marker)
    instruction = entire_conv[start_index + len(start_marker):end_index].strip()
    prompt = entire_conv[start_index:end_index + len(end_marker)].strip()
    original_answer = entire_conv[end_index + len(end_marker): len(entire_conv) - len(end_tag) - 1].strip()
    original_instructions.append(instruction)
    ground_truth_answer.append(original_answer)
    print("encoding prompt number {}...".format(ri+1))
    input_ids = restored_tokenizer.encode(prompt, return_tensors="pt").to('cuda')
    print("generating response number {} ...".format(ri+1))
    outputs = deepspeed_peft_model.generate(input_ids=input_ids, 
        max_new_tokens=256,
        do_sample=True,
    )
    pred = restored_tokenizer.decode(outputs[0])
    extracted_pred = pred[pred.find(end_marker) + len(end_marker): len(pred)].strip()
    predictions.append(pred)
    extracted_answers.append(extracted_pred)
    print("Instruction: \n")
    print(instruction)
    print()
    print("Ground truth answer: \n")
    print(original_answer)
    print()
    print("Generated answer: \n")
    print(extracted_pred)
    print()
    print("====================")
    if ri == 5:
        break

#"{}/{}".format(base_dir, checkpoint_number)
output_file_name = "generated_answers_peft_{}_{}".format(base_dir, checkpoint_number)

pred_dataframe = pd.DataFrame(zip(original_instructions, ground_truth_answer, extracted_answers, predictions), columns=["Instructions", "Ground truth answers", "Predicted answers", "Full generated answers"])

pred_dataframe.to_csv("../data/{}.csv".format(output_file_name), sep="\t", index=None)

e_time = time.time()

print("Finished generation in {} seconds".format(e_time - s_time))

encoding prompt number 1...
generating response number 1 ...


/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Instruction: 

I just set up a fresh galaxy installation, following the instructions on: @link https://galaxyproject.org/admin/get-galaxy/ . But once I try to install a tool I get the following error:
Error cloning repository: [Errno 2] No such file or directory: ‘hg’: ‘hg’
It seems to always be the same error with every tool. Can someone here help me out?
EDIT: Found it. Seems I needed to install mercurial. But it seems this is never mentioned in the installation docs?

Ground truth answer: 

I have added a note to our toolshed installation tutorial that should clarify this. @link https://galaxyproject.org/admin/tools/add-tool-from-toolshed-tutorial/#connect-your-galaxy-to-a-tool-shed
Thanks for bringing it up!

Generated answer: 

Welcome, @user
The problem with cloning repository is due to disk space usage on your server. Allowing your server to run out of space can cause the process to fail.
Galaxy can run out of space for temporary work items while the server is waiting for a job 